In [ ]:
import numpy as np
import numba
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from scipy.integrate import solve_ivp
from space_traj_opt.plotting import plot

In [ ]:
thrust = 210000.0
g0 = 1.61544
STANDARD_GRAV = 9.81
Isp = 265.2 
a0 = -0.001
b0 = 3.0
params = (thrust, g0, Isp, a0, b0)

x0 = np.array([0,0,0,0,50000])

In [ ]:
@numba.njit
def lts_control(t, x , params):
    a = params[3]
    b = params[4]
    return np.arctan(a*t+ b)

@numba.njit
def dynamics(t, x, params):
    thrust = params[0]
    g = params[1]
    Isp = params[2]
    u = lts_control(t, x, params)

    cos_theta  = np.cos(u)
    sin_theta  = np.sin(u)
    dx = np.zeros_like(x)
    dx[0] = x[2]
    dx[1] = x[3]
    dx[2] =  (thrust * cos_theta ) / x[4] 
    dx[3] =  (thrust * sin_theta )  / x[4] - g
    dx[4] =  -thrust / STANDARD_GRAV / Isp
    return dx

In [ ]:
def mass_limit(t, x, params):
    return x[4]

mass_limit.terminal = True
mass_limit.direction = -1  # Terminate when value crosses zero

In [ ]:
sol = solve_ivp(
    dynamics, 
    t_span=[0, 500], 
    y0=x0,    args=(params,), 
    events=mass_limit)

In [ ]:
mdot = thrust / STANDARD_GRAV / Isp

t_max = 50000 / mdot

In [ ]:
# Fin a0, and b0 Such that 
a0 = -0.001
b0 = 3.0
t_guess = 500
terminal_x = [None, 1.8500e+05, 1.6270e+03, 0, None]
t_normalized = 1
decision_var_init = [a0, b0, t_normalized]

arguments = [
    x0,
    (thrust, g0, Isp)
]
def problem (decision_var,params):
    x0 = params[0]
    thrust, g0, Isp = params[1]
    args= (thrust, g0, Isp, decision_var[0], decision_var[1])
    sol = solve_ivp(
    dynamics, 
        t_span=[0, decision_var[2] * t_guess], 
        y0=x0, 
        args=(args,)
        )
    xf_x, xf_y, xf_vx, xf_vy, mf = sol.y[:,-1]
    cost =  100*((xf_y - 1.8500e+05) / 1.8500e+05)**2 + \
            150*((xf_vx - 1.6270e+03)/1.6270e+03)**2 + \
            10*(xf_vy/1.6270e+03)**2 + 0.001*decision_var[2]**2
    return cost

bounds = [
    (-1,1),
    (-5,5),
    (None, t_max),
    ]

In [ ]:
result = minimize(problem, decision_var_init, method='SLSQP', bounds=bounds, args=arguments)
result

In [ ]:
a_opt, b_opt, t_normalized_opt = result.x
params_opt = (thrust, g0, Isp, a_opt, b_opt)


In [ ]:
a_opt, b_opt

In [ ]:
t_normalized_opt*t_guess

In [ ]:
t_eval = np.arange(0.0, t_normalized_opt*t_guess, 5)
t_eval = np.append(t_eval, t_normalized_opt*t_guess)
sol = solve_ivp(
    dynamics, 
    t_span=[0, t_normalized_opt*t_guess], 
    y0=x0,    
    t_eval=t_eval,
    args=(params_opt,), 
    events=mass_limit)

In [ ]:
u_opt = lts_control(t_eval, sol.y, params_opt)

In [ ]:
plot( sol.y[0], [sol.y[1]], title="Alt vs downrange", xlabel="X", ylabel="Y", )

In [ ]:
plot(
    sol.t, [u_opt], y2 = [np.tan(u_opt)],
    title="Time vs Pitch steering", 
    xlabel="Time", 
    ylabel=("theta", "tan_theta"),
    trace_names=("theta", "tan_theta")
    )

In [ ]:
plot(
    sol.t,[sol.y[0], sol.y[1]], y2 = [sol.y[2], sol.y[3]],
    title="Time vs States", 
    xlabel="Time", 
    ylabel=("Pos", "Vel"),
    trace_names=("pos_x", "pos_y", "vel_x", "vel_y")
    )

In [ ]:
plot( sol.t, [sol.y[4]], title="time vs Mass", xlabel="Time", ylabel="Mass")